In [1]:
import sklearn
import statsmodels
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
df=pd.read_csv('train (1).csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [21]:
df_titanic=df[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Survived']]

In [22]:
#ssplitting data

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
X=df_titanic.drop(columns='Survived')
y=df_titanic['Survived']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = .8, random_state=42)

In [26]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [27]:
cat_columns=['Sex','Embarked','Pclass']
num_columns=['Age','SibSp','Parch','Fare']

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder,MinMaxScaler,StandardScaler,RobustScaler

In [29]:
numerical_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),#nama dari langkahnya
    ('scaler',RobustScaler())
])

categorical_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder())
])

In [30]:
scaler=RobustScaler()

In [31]:
#hasil tahap satu
X_train_num=numerical_pipeline.fit_transform(X_train[num_columns])

In [32]:
X_train_num

array([[ 1.23085799,  0.        ,  0.        ,  0.62218383],
       [-0.49991124,  0.        ,  0.        , -0.06441639],
       [ 0.19239645,  0.        ,  0.        , -0.28922259],
       ...,
       [ 0.88470414,  2.        ,  0.        , -0.01532226],
       [-1.19221893,  1.        ,  2.        ,  4.67533998],
       [-0.6537574 ,  0.        ,  1.        ,  2.7833134 ]])

In [33]:
X_train_cat=categorical_pipeline.fit_transform(X_train[cat_columns])

In [34]:
X_train_cat.toarray()

array([[0., 1., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 0., 1.],
       ...,
       [0., 1., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 1., 0., 0.],
       [0., 1., 0., ..., 1., 0., 0.]])

In [35]:
from sklearn.compose import ColumnTransformer# buat ngegabungin hasil num dengan hasil cat

semua data baru harus melalui step yang sama dengan yang dilalui oleh X_train
atau harus dalam format yang sama

In [36]:
preprocessor=ColumnTransformer([
    ('numeric',numerical_pipeline,num_columns),
    ('categorical',categorical_pipeline,cat_columns)
])

In [37]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [38]:
X_train.isna().sum()#awalnya kaya gini

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [39]:
X_train_prep=preprocessor.fit_transform(X_train)

In [41]:
X_test_prep=preprocessor.fit_transform(X_test)

In [42]:
from sklearn.ensemble import RandomForestClassifier

In [43]:
RF=RandomForestClassifier()

In [44]:
RF.fit(X_train_prep,y_train)

RandomForestClassifier()

In [46]:
RF.score(X_test_prep,y_test)

0.770949720670391

In [47]:
pipeline=Pipeline([
    ('prep',preprocessor),
    ('algo',RandomForestClassifier())
])

In [48]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [49]:
model=RandomForestClassifier()

In [50]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                  

In [51]:
pipeline.score(X_test,y_test)

0.8100558659217877

In [ ]:
numerical_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='mean')),#nama dari langkahnya, imputer buat ngisi nan value
    ('scaler',RobustScaler())
])

categorical_pipeline=Pipeline([
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder())
])
preprocessor=ColumnTransformer([
    ('numeric',numerical_pipeline,num_columns),
    ('categorical',categorical_pipeline,cat_columns)
])
pipeline=Pipeline([
    ('prep',preprocessor),
    ('algo',RandomForestClassifier())
])

In [52]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [65]:
data_eric={
    'nama':'eric',
    'Pclass':2,
    'Sex':'male',
    'Age':28,
    'SibSp':2,
    'Parch':2,
    'Fare':58,
    'Embarked':'5'
}

In [66]:
eric=pd.DataFrame(data_eric,index=[1])

In [67]:
eric

,nama,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,eric,2,male,28,2,2,58,5


In [68]:
pipe_rf=Pipeline([
    ('prep',preprocessor),
    ('algo',RandomForestClassifier())
])

In [69]:
pipe_rf.fit(X_train,y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                  

In [70]:
pipe_rf.predict(eric)[0]

ValueError: Found unknown categories ['5'] in column 1 during transform